# <span style="color:red">Important</span>

## We MUST ALWAYS import  <span style="color:red">*ROSPY*</span> before <span style="color:blue">PyTORCH</span>. Otherwise you WILL get an error

In [ ]:
import rospy   # Imports ROS fpr Python
from geometry_msgs.msg import Twist, Point, Quaternion   # Import  libraries necessary to control the engines
from nav_msgs.msg import Odometry   # To obtain information of how long you have moved

## Now, it is safe to import PyTORCH

In [ ]:
import torch

## Import Camera 

In [ ]:
import getpass   # To interact with the terminal automatically
import os

# These three lines are used to Reset the Camera. 
# This allows you to re-run the script (after restarting the kernel) without having to restart the Jetson.
# If you don't run these three lines below, you will get an error saying your camera is not ready
password = 'jetbot'   # If your jetson has a different password, replace it with your password.
command = "sudo -S systemctl restart nvargus-daemon"
os.system('echo %s | %s' % (password, command))

# Here we import the camera & required libraries
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=640, height=480, capture_fps=10)
camera.running = True
print("jetcam ready!")

## Import Lidar Libraries

In [ ]:
import rplidar  # To use the rpLidar A1
import getpass   # For the same reason as above
from threading import Thread
from rplidar import RPLidar
import rplidar
import os

# Reset Lidar. Same idea as the camera
password = 'jetbot'
command = "sudo -S chmod 666 /dev/ttyUSB1" #can be any command but don't forget -S as it enables input from stdin
os.system('echo %s | %s' % (password, command))
print("lidar ready!")

## Lidar thread

In [ ]:
# Define what angles you want to measure. In this case, 0, 10, 20, ..., 360
anglesToRead = [x*10 for x in range(36)]

# This variable will contain ALL the readings from the lidar (all angles between 0 and 360)
Real_Readings = {x:1 for x in [x for x in range(360)]} 

In [ ]:
def readLidarThread():
    
    global lidar, Real_Readings
    
    while True:
        
        for scan in lidar.iter_scans(scan_type='express', min_len = 100, max_buf_meas=False):
            RAW_Readings = {int(x[1]): int(x[2]) for x in list(scan)}
            
            # we clean the RAW readings as they might be repeated or incomplete.
            # Remember: when we call iter_scans one time, we are not reading the angles in order. 
            # Also, this command skips some angles and re-scans other angles in a single spin
            for key,value in RAW_Readings.items(): 
                Real_Readings[key]=value

In [ ]:
# Initialize the lidar object
lidar = RPLidar('/dev/ttyUSB1')
lidar.motor_speed = rplidar.MAX_MOTOR_PWM

# Start the Lidar Thread
lidarThread = Thread(target = readLidarThread)
lidarThread.start()

In [ ]:
# Print the lidar readings every second, then clear the output
from IPython import display
import time

# Print every second for 20 seconds
for _ in range (20):  
    # Clear the print output
    display.clear_output(wait=True)
    
    # Obtain the lidar readings we actually care about (defined by the angles in anglesToRead)
    Readings_we_care_about = {k:v for k,v in Real_Readings.items() if k in anglesToRead}
    
    # To print all the readings vertically (easier for a person to read)
    for key, value in Readings_we_care_about.items():
        print(key, ' : ', value)
    time.sleep(1)

## Camera Thread

In [ ]:
def execute(change):
    # Obtain a new frame
    image = change['new']  
    
    # Convert it to an RGB image
    image_w.value = bgr8_to_jpeg(image[::-1, ::-1, :])  # -1 is to flip image horiz. or vert. Remove if necessary.

In [ ]:
# Create the widget
image_w = ipywidgets.Image(format='jpeg', width=640, height=480)

# Show the widget
display(image_w)

# Start camera and update the widget in the thread (infinitely)
execute({'new': camera.value})
camera.observe(execute, names='value')

## Movement / Rotation Thread

## Below are some necessary functions to obtain an accurate reading from the stepper engines.

In [ ]:
from math import radians, copysign, sqrt, pow, pi, atan2
import numpy
import math

dx,dy = 0,0
real_x,real_y = 0,0
moved_distance=0

_AXES2TUPLE = {
    'sxyz': (0, 0, 0, 0), 'sxyx': (0, 0, 1, 0), 'sxzy': (0, 1, 0, 0),
    'sxzx': (0, 1, 1, 0), 'syzx': (1, 0, 0, 0), 'syzy': (1, 0, 1, 0),
    'syxz': (1, 1, 0, 0), 'syxy': (1, 1, 1, 0), 'szxy': (2, 0, 0, 0),
    'szxz': (2, 0, 1, 0), 'szyx': (2, 1, 0, 0), 'szyz': (2, 1, 1, 0),
    'rzyx': (0, 0, 0, 1), 'rxyx': (0, 0, 1, 1), 'ryzx': (0, 1, 0, 1),
    'rxzx': (0, 1, 1, 1), 'rxzy': (1, 0, 0, 1), 'ryzy': (1, 0, 1, 1),
    'rzxy': (1, 1, 0, 1), 'ryxy': (1, 1, 1, 1), 'ryxz': (2, 0, 0, 1),
    'rzxz': (2, 0, 1, 1), 'rxyz': (2, 1, 0, 1), 'rzyz': (2, 1, 1, 1)}

_TUPLE2AXES = dict((v, k) for k, v in _AXES2TUPLE.items())

# epsilon for testing whether a number is close to zero
_EPS = numpy.finfo(float).eps * 4.0

# axis sequences for Euler angles
_NEXT_AXIS = [1, 2, 0, 1]

def euler_from_quaternion(quaternion, axes='sxyz'):
    """Return Euler angles from quaternion for specified axis sequence.
    >>> angles = euler_from_quaternion([0.06146124, 0, 0, 0.99810947])
    >>> numpy.allclose(angles, [0.123, 0, 0])
    True
    """
    return euler_from_matrix(quaternion_matrix(quaternion), axes)
#     return euler_from_matrix(quaternion_matrix(quaternion), axes)[2]

def euler_from_matrix(matrix, axes='sxyz'):
    """Return Euler angles from rotation matrix for specified axis sequence.
    axes : One of 24 axis sequences as string or encoded tuple
    Note that many Euler angle triplets can describe one matrix.
    >>> R0 = euler_matrix(1, 2, 3, 'syxz')
    >>> al, be, ga = euler_from_matrix(R0, 'syxz')
    >>> R1 = euler_matrix(al, be, ga, 'syxz')
    >>> numpy.allclose(R0, R1)
    True
    >>> angles = (4.0*math.pi) * (numpy.random.random(3) - 0.5)
    >>> for axes in _AXES2TUPLE.keys():
    ...    R0 = euler_matrix(axes=axes, *angles)
    ...    R1 = euler_matrix(axes=axes, *euler_from_matrix(R0, axes))
    ...    if not numpy.allclose(R0, R1): print axes, "failed"
    """
    try:
        firstaxis, parity, repetition, frame = _AXES2TUPLE[axes.lower()]
    except (AttributeError, KeyError):
        _ = _TUPLE2AXES[axes]
        firstaxis, parity, repetition, frame = axes

    i = firstaxis
    j = _NEXT_AXIS[i+parity]
    k = _NEXT_AXIS[i-parity+1]

    M = numpy.array(matrix, dtype=numpy.float64, copy=False)[:3, :3]
    if repetition:
        sy = math.sqrt(M[i, j]*M[i, j] + M[i, k]*M[i, k])
        if sy > _EPS:
            ax = math.atan2( M[i, j],  M[i, k])
            ay = math.atan2( sy,       M[i, i])
            az = math.atan2( M[j, i], -M[k, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2( sy,       M[i, i])
            az = 0.0
    else:
        cy = math.sqrt(M[i, i]*M[i, i] + M[j, i]*M[j, i])
        if cy > _EPS:
            ax = math.atan2( M[k, j],  M[k, k])
            ay = math.atan2(-M[k, i],  cy)
            az = math.atan2( M[j, i],  M[i, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2(-M[k, i],  cy)
            az = 0.0

    if parity:
        ax, ay, az = -ax, -ay, -az
    if frame:
        ax, az = az, ax
    return ax, ay, az

def quaternion_matrix(quaternion):
    """Return homogeneous rotation matrix from quaternion.
    >>> R = quaternion_matrix([0.06146124, 0, 0, 0.99810947])
    >>> numpy.allclose(R, rotation_matrix(0.123, (1, 0, 0)))
    True
    """
    q = numpy.array(quaternion[:4], dtype=numpy.float64, copy=True)
    nq = numpy.dot(q, q)
    if nq < _EPS:
        return numpy.identity(4)
    q *= math.sqrt(2.0 / nq)
    q = numpy.outer(q, q)
    return numpy.array((
        (1.0-q[1, 1]-q[2, 2],     q[0, 1]-q[2, 3],     q[0, 2]+q[1, 3], 0.0),
        (    q[0, 1]+q[2, 3], 1.0-q[0, 0]-q[2, 2],     q[1, 2]-q[0, 3], 0.0),
        (    q[0, 2]-q[1, 3],     q[1, 2]+q[0, 3], 1.0-q[0, 0]-q[1, 1], 0.0),
        (                0.0,                 0.0,                 0.0, 1.0)
        ), dtype=numpy.float64)

def calculateDistance(x0,y0,x1,y1):
    return math.sqrt((x0-x1)**2+(y0-y1)**2)

### This function is used to calculate the angle with respect to the original orientation. Otherwise, the angle returned by default by ROS is inconsistent with our Unity env

In [ ]:
def callback(data):
    global dx,dy,angle
    dx = data.pose.pose.position.x
    dy = data.pose.pose.position.y
    
    quat = [data.pose.pose.orientation.x,
            data.pose.pose.orientation.y,
            data.pose.pose.orientation.z,
            data.pose.pose.orientation.w]
    
    angle = math.degrees(euler_from_quaternion(quat))

In [ ]:
def measurementThread():
    running_thread = True
    
    # Create a suscribe to Odometry node using callback function   
    odomSuscriber = rospy.Subscriber('/odom', Odometry, callback, queue_size=1)
    
    # Update odom measurements forever
    rospy.spin()

In [ ]:
def Move(distance=0.12,speed=0.4):
    direction = -1 if speed < 0 else 1
    global dx,dy,angle
    global real_angle, real_x, real_y
    global moved_distance, movements
    global r
    
    move = Twist()
    move.linear.x=speed
    
    init_x = dx
    init_y = dy
    
    while calculateDistance(init_x,init_y,dx,dy) < distance:
        pub.publish(move)
        r.sleep()
    moved_distance = calculateDistance(init_x,init_y,dx,dy)*direction
    print('/nmoved: ', moved_distance)

In [ ]:
def Rotate(degrees=10, direction='left'):
    global dx,dy,angle
    global real_angle
    
    # Angle symbol
    direct = 1 if direction=='left' else -1
    
    # Create msg to communicate with ROS
    rotate = Twist()
    
    # Define speed and direction of rotation
    rotate.angular.z = direct*0.175

    # Rotate and calculate how much it rotated
    init_angle = angle
    while abs(init_angle-angle) < degrees:#*0.9:
        pub.publish(rotate)
        r.sleep()
    # Update real_angle
    rotated=direct*abs(init_angle-angle)
    real_angle-=rotated
    print("rotated = ", rotated)
    
    # Make sure real_angle is between 0-360
    if real_angle/360>1:
        real_angle-=360
    elif real_angle<0:
        real_angle+=360

In [ ]:
def update_coordinates():
    
    global moved_distance
    global real_angle, real_x, real_y
    
    if real_angle == 0: # Moved Forward ==> Y++
        real_y+=moved_distance
        
    elif real_angle == 90: # Moved Right ==> X++
        real_x+=moved_distance
        
    elif real_angle == 180: # Moved Backward ==> Y--
        real_y-=moved_distance
        
    elif real_angle == 270: # Moved Left ==> X--
        real_x-=moved_distance
        
    else:  # ==> Calculate X,Y components 
        delta_x = moved_distance*math.sin(math.radians(real_angle))
        delta_y = moved_distance*math.cos(math.radians(real_angle))
               
        real_x += delta_x
        real_y += delta_y

In [ ]:
# Create Node to communicate with ROS
rospy.init_node('sub_odom')
dx,dy,angle = 0,0,0
real_angle, real_x, real_y = 0,0,0
moved_distance = 0
rotations,movements = [],[]

In [ ]:
# Create a publisher to control movement
pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)

# Run Thread
running_thread = False
if not running_thread:
    thread = Thread(target = measurementThread)
    thread.start()

In [ ]:
Move(distance=0.10, speed=-0.15)  # meters, m/s
update_coordinates()
print(f'new coordinates: angle={real_angle}, y={real_y}, x={real_x}')

In [ ]:
Rotate(degrees=10, direction='left')
print(f'new coordinates: angle={real_angle}, y={real_y}, x={real_x}')